In [22]:
import os
import pandas as  pd 
import numpy as np 
import requests, zipfile

In [23]:
train=pd.read_csv('/kaggle/input/wifi-data/Train (17).csv')
test=pd.read_csv('/kaggle/input/wifi-data/Test (18).csv')
tt=pd.read_csv('/kaggle/input/wifi-data/TrainTargets.csv')

In [24]:
def preprocess(data,is_train=True):

  dep=data['Deployment'].unique()
  rows=[]
  for dd in dep:
    df=data[data.Deployment==dd]
    
    toxp_map=dict()
    rssi_map=dict()
    flag=0
    for i, j in df.iterrows():
        if('STA' in j[0]):
          rssi_map[j[0]]={f'AP{i}':j[i] for i in range(1,10)}
        else:
          toxp_map[j[0]]={(j.index[i],'STA'+str(int(j[i]))) for i in range(1,10)}

    
    for exp,elems in toxp_map.items():
      rows1=[]
      for e in elems:
        row=[dd,exp]
        if(e[-1]!='STA0'):
          row.extend([e[0],e[1],rssi_map[e[-1]][e[0]]])
          rows.append(row)
  

      #rows.extend(rows1)
    
  data=pd.DataFrame(rows)
  data.columns = ['Deployment','TXOP','mainAP','mainSTA','mainRSSI']
  data['ID']='Deployment'+data['Deployment'].astype(str)+'_'+data['TXOP']+'_'+data['mainAP']+'_'+data['mainSTA']
  if(is_train):
    tt=pd.read_csv('/kaggle/input/wifi-data/TrainTargets.csv')
    data=pd.merge(data,tt,on='ID')
  
  
  return data

In [44]:
tr=preprocess(train)

In [45]:
y=tr["Target"]

In [25]:
trainn=preprocess(train)
test=preprocess(test,False)

In [26]:
for i in range(trainn.shape[0]):
    if trainn.loc[i,"mainRSSI"]>10000000:
        trainn.loc[i,"mainRSSI"]=10000000
for i in range(test.shape[0]):
    if test.loc[i,"mainRSSI"]>10000000:
        test.loc[i,"mainRSSI"]=10000000

In [27]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

trainn['mainSTA']= LE.fit_transform(trainn['mainSTA'])
test['mainSTA']= LE.transform(test['mainSTA'])

trainn['TXOP']= LE.fit_transform(trainn['TXOP'])
test['TXOP']= LE.transform(test['TXOP'])

trainn['mainAP']= LE.fit_transform(trainn['mainAP'])
test['mainAP']= LE.transform(test['mainAP'])





In [28]:
!pip install lazypredict

In [29]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

In [30]:
col=['Deployment', 'TXOP', 'mainAP', 'mainSTA', 'mainRSSI']

In [38]:
trainn = mms.fit_transform(trainn)
trainn=pd.DataFrame(trainn, columns=col)
testt = mms.transform(test.drop("ID", axis=1))
test=pd.DataFrame(testt, columns=col)


In [1]:
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle
import numpy as np
import mock
from sklearn.model_selection import train_test_split
X=trainn[['mainRSSI']]
y=y
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2,  random_state=42)
reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(X_train, X_test, y_train, y_test)

NameError: name 'trainn' is not defined

In [ ]:
print(models)

In [ ]:
#Making submission
ss=test[['ID']]
ss['Target']=np.mean(predictions,0)
ss
ss['Target']=ss['Target'].astype(np.float16)#Otherwise file will be too large
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(ss, "sub1")

****